In [26]:
from copy import copy

        

In [110]:
RITUAL = 3
CULTIST_HP = 48

class Game:
    def __init__(self):
        self.turn = 1
        self.todraw = 5
        
        self.cultisthp = 15
        
        self.hplost = 0
        self.energy = 3
        self.block = 0
        self.deck = tuple(sorted(['strike']*5 + ['block']*5))
        self.hand = tuple()
        self.discard = tuple()
        
    def __hash__(self):
        assert self.deck == tuple(sorted(self.deck))
        assert self.hand == tuple(sorted(self.hand))
        if self.discard != tuple(sorted(self.discard)):
            print(self.discard)
            print(tuple(sorted(self.discard)))
            assert self.discard == tuple(sorted(self.discard)) 
        
        return hash((
            self.turn, self.cultisthp, self.hplost, self.energy, self.block,
            self.deck, self.hand, self.discard
        ))
    
    def __str__(self):
        return str(self.__dict__)
    

# f(gamedetails) -> tuple<gamedetails>
def FAN_OUT(game):
    if game.todraw:
        self = copy(game)
        self.todraw -= 1
        
        if not self.deck:
            self.deck = self.discard
            self.discard = tuple()
        
        return tuple(
            move_deck_to_hand(copy(self), x)
            for x in range(0, len(self.deck))
        )
    
    if game.energy == 0:
        return (endturn(copy(game)),)
    
    return tuple(
        playcard(copy(game), cardname)
        for cardname in game.hand
    )
    

# below are actions
#   f(gamestate, *params) -> modified gamestate
def move_deck_to_hand(new, x):
    new.hand = tuple(sorted(new.hand + (new.deck[x],)))
    new.deck = tuple(new.deck[:x]) + tuple(new.deck[x+1:])
    return new

def endturn(new):
    new.discard = tuple(sorted(new.discard + new.hand))
    new.hand = tuple()
    
    if new.turn > 1:
        damage = (new.turn-1)*RITUAL + 6
        blocked = min(damage, new.block)
        new.hplost += damage - blocked
    
    new.todraw = 5
    new.energy = 3
    new.block = 0
    new.turn += 1
    return new
    
def playcard(new, name):
    x = new.hand.index(name)
    new.hand = tuple(new.hand[:x]) + tuple(new.hand[x+1:])
    
    if name == 'strike':
        new.cultisthp -= 6
        new.energy -= 1
        
    if name == 'block':
        new.block += 5
        new.energy -= 1
        
    new.discard = tuple(sorted(new.discard + (name,)))
    return new


g = Game()
print(g)

for c in FAN_OUT(g):
    print(bin(hash(c)))

{'turn': 1, 'todraw': 5, 'cultisthp': 15, 'hplost': 0, 'energy': 3, 'block': 0, 'deck': ('block', 'block', 'block', 'block', 'block', 'strike', 'strike', 'strike', 'strike', 'strike'), 'hand': (), 'discard': ()}
0b101001111000111101001000000011110011100100110100111101100010101
0b101001111000111101001000000011110011100100110100111101100010101
0b101001111000111101001000000011110011100100110100111101100010101
0b101001111000111101001000000011110011100100110100111101100010101
0b101001111000111101001000000011110011100100110100111101100010101
-0b101010111100011010001011001011101000111000011101010111110011011
-0b101010111100011010001011001011101000111000011101010111110011011
-0b101010111100011010001011001011101000111000011101010111110011011
-0b101010111100011010001011001011101000111000011101010111110011011
-0b101010111100011010001011001011101000111000011101010111110011011


In [118]:

# hash : gamedetails | tuple<hash> | int hploss
TREE = {}  

# hash : status
QUEUE = {}


def process(gamehash):
    self = TREE[gamehash]
    assert type(self) == Game
    
    if self.cultisthp <= 0:
        print('win turn', self.turn, 'taking', self.hplost, 'damage')
        return 'win', self.turn, self.hplost
    
    if self.hplost >= 20:
        print('lost turn', self.turn, 'with', self.cultisthp, 'remaining')
        return 'loss', self.turn, self.cultisthp
    
    children = {hash(c):c for c in FAN_OUT(self)}
    for key, child in children.items():
        TREE[key] = child
        QUEUE[key] = True
        
    return tuple(children.keys())

root = Game()
TREE[hash(root)] = root
TREE[hash(root)] = process(hash(root))


for _ in range(5000):
    for key in QUEUE.keys():
        break
    try:
        del QUEUE[key]
    except KeyError:
        print(key)
        print(key in QUEUE.keys())
        raise KeyError
    
    TREE[key] = process(key)

# TREE, QUEUE

win turn 1 taking 0 damage
win turn 1 taking 0 damage
win turn 1 taking 0 damage
win turn 2 taking 0 damage
win turn 2 taking 0 damage
win turn 2 taking 0 damage
win turn 2 taking 0 damage
win turn 2 taking 0 damage
win turn 2 taking 0 damage
win turn 2 taking 0 damage
win turn 2 taking 0 damage
win turn 2 taking 0 damage
win turn 2 taking 0 damage
win turn 2 taking 0 damage
win turn 2 taking 0 damage
win turn 2 taking 0 damage
win turn 2 taking 0 damage
win turn 2 taking 0 damage
win turn 2 taking 0 damage
win turn 2 taking 0 damage
win turn 2 taking 0 damage
win turn 3 taking 4 damage
win turn 3 taking 4 damage
win turn 3 taking 4 damage
win turn 3 taking 4 damage
win turn 3 taking 4 damage
win turn 3 taking 0 damage
win turn 3 taking 0 damage
win turn 3 taking 0 damage
win turn 3 taking 0 damage
win turn 3 taking 0 damage
win turn 3 taking 4 damage
win turn 3 taking 4 damage
win turn 3 taking 4 damage
win turn 3 taking 4 damage
win turn 3 taking 0 damage
win turn 3 taking 0 damage
w

KeyError: 

({-3919975710908060937: (-3602035520367175373, -2323029743063383421),
  -3602035520367175373: (-7619287883176128461, 4094420782382501151),
  -2323029743063383421: (4094420782382501151, -2593668302727538189),
  -7619287883176128461: (5026433690224480359, -3292112352714727945),
  4094420782382501151: (-3292112352714727945, -3675984573775132493),
  -2593668302727538189: (-3675984573775132493, -4329245500794251661),
  5026433690224480359: (-6254131865225671217, 3511817536782736651),
  -3292112352714727945: (3511817536782736651, -7875787301520834049),
  -3675984573775132493: (-7875787301520834049, -332132940354778525),
  -4329245500794251661: (-332132940354778525, 7933850593413903407),
  -6254131865225671217: (4945262684232087147, 784138157387092539),
  3511817536782736651: (784138157387092539, 6767432528101452631),
  -7875787301520834049: (6767432528101452631, 3296901242516528879),
  -332132940354778525: (3296901242516528879, -7250818612274538221),
  7933850593413903407: (-7250818612274538

In [71]:
print(TREE[-332132940354778525])

{'turn': 0, 'todraw': 1, 'cultisthp': 10, 'hplost': 0, 'energy': 3, 'block': 0, 'deck': ('block', 'block', 'block', 'block', 'strike', 'strike'), 'hand': ('block', 'strike', 'strike', 'strike'), 'discard': ()}
